In [69]:
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
import re
import string
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import csv
import time
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize
from nltk import word_tokenize
nltk.download('vader_lexicon')
nltk.download('punkt')
from sklearn.linear_model import LogisticRegression

#import code.auth_and_scrape as get_tweets
#import code.preparation as prep
#import code.visualization as viz

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Twitter API Authentication

In [2]:
# Authentication
consumerKey = 'API Key'
consumerSecret = 'API Secret'
accessToken = 'Acccess Token'
accessTokenSecret = 'Access Secret'
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

To CSV Function

In [3]:
def append_to_csv(tweets, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in tweets:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. ID
        id = tweet.id

        # 2. Time created
        created_at = tweet.created_at

        # 4. Language
        lang = tweet.lang

        # 5. Tweet metrics
        retweet_count = tweet.retweet_count
        like_count = tweet.favorite_count

        # 6. Tweet text
        text = tweet.text
        
        # Assemble all data in a list
        res = [id, created_at, lang, like_count, retweet_count, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

Function to scrape a chosen amount of tweets with a chosen keyword and append to CSV file.

In [23]:
def scrape_and_place():
    #Ask for keyword with which to scrape.
    keyword = input('Please enter keyword or hastag to search: ')
    #Ask for number of tweets desired to scrape.
    tweet_num = int(input('Please enter how many tweets to analyze (If greater than 900, must be multiple of 900): '))
    #Ask for date at which access was requested as the scrape can only occur on tweets made within seven days prior to requested date.
    date_req = input('Please enter the date from which you request YYYY/MM/DD: ')
    date_req = date_req.split('/')
    j = 0
    #Only 900 tweets can be scraped every 15min so if desired amount is greater than 900 this will create a wait time to pull every 15 min.
    if tweet_num > 900:
        for i in range(0, tweet_num//900):
            if j == 7:
                break
            elif j != 7:
                tweets = tweepy.Cursor(api.search_tweets, q=keyword+' lang:en -filter:retweets', until=date_req[0] + '-' + date_req[1] + '-' + str((int(date_req[2]) - j))).items(900)
            j += 1
            append_to_csv(tweets, fileName='data.csv')
            time.sleep(900)

    elif tweet_num <= 900 & tweet_num > 0:
        for i in range(0, tweet_num):
            tweets = tweepy.Cursor(api.search_tweets, q=keyword+' lang:en -filter:retweets', until=date_req[0] + '-' + date_req[1] + '-' + date_req[2]).items(tweet_num)
            append_to_csv(tweets, fileName='data.csv')
        

In [5]:
scrape_and_place()

# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900


In [50]:
df = pd.read_csv('data/data.csv')
df.head()

,id,created_at,language,favorites,retweets,text
0,1462933594754670600,2021-11-22 23:58:43+00:00,en,0,0,@DestinyTheGame all id like is to be at least ...
1,1462933098887266308,2021-11-22 23:56:45+00:00,en,0,0,@DestinyTheGame Can someone please explain to ...
2,1462932969086087176,2021-11-22 23:56:14+00:00,en,1,0,@techaftmidnight @GoogleStadia @DestinyTheGame...
3,1462932844418842632,2021-11-22 23:55:44+00:00,en,1,0,Going live in 5 minutes!! Let's go GOD SQUAD!!...
4,1462932760515985420,2021-11-22 23:55:24+00:00,en,0,0,"@A_dmg04 @DestinyTheGame Dear Eva, please brin..."


In [51]:
df.favorites.describe()

count    6300.000000
mean        9.520635
std        90.900910
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max      3879.000000
Name: favorites, dtype: float64

Cleaning of Tweets

In [52]:
df.text[1]

'@DestinyTheGame Can someone please explain to me why I (top 7-10%ish player) keep getting teamed with people in the… https://t.co/fREoFm7Emb'

This tweet has a ton of punctuation and it even includes a url. In order for our sentiment analyzer to read and understand this, we must remove the all of this and even lowercase every letter. Here A function has been create to do this.

In [53]:
#Function to clean tweets of urls, punctuation and uppercase letters
def clean_tweets(dataframe):

    #Lambda function to remove urls from tweets providing links
    remove_url = lambda tweet: re.sub(r'http\S+', '', tweet)

    #Lambda function to remove punctuation from tweets
    remove_punct = lambda tweet: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+://\S+)"," ", tweet)

    #Remove urls and punctuation from text column
    dataframe["clean_text"] = dataframe.text.map(remove_url).map(remove_punct)

    #Reduce text to lowercase format
    dataframe["clean_text"] = dataframe.clean_text.str.lower()

    return dataframe

In [54]:
df = clean_tweets(df)

A function to create a Sentiment Analysis using TextBlob and SentimentIntensityAnalyzer.

In [55]:
#Calculating Negative, Positive, Neutral and Compound 
def create_sentiment(dataframe):
    df_probs = pd.DataFrame()

    #create dataframe of subjectivity and subjectivity polarity of tweets
    df_probs[['polarity', 'subjectivity']] = dataframe['clean_text'].apply(lambda text: pd.Series(TextBlob(text).sentiment))

    #Scores each tweet based on the subjectivity polarity
    for index, row in dataframe['text'].iteritems():

        #Provides negative, neutral, positive and compound scores for the tweet in that row
        score = SentimentIntensityAnalyzer().polarity_scores(row)

        #extracts negative and positive scores
        neg = score['neg']
        pos = score['pos']
        
        #creates binary values based on which sentiment score is higher 
        if neg > pos:
            df_probs.loc[index, 'sentiment'] = 0
        elif pos > neg:
            df_probs.loc[index, 'sentiment'] = 1
        else:
            df_probs.loc[index, 'sentiment'] = 0
            
    return df_probs

In [56]:
df_probs = create_sentiment(df)

Feature Engineering Positive Reinforcement Variable

In [57]:
#Creates a dataframe of the positive tweets and matches their sentiment values
def create_pos_df(dataframe, dataframe_probs):

    #Grabs text from original dataframe 
    dataframe.reset_index(drop=True, inplace=True)
    df_pos = df[['text']].copy(deep=True)

    #Grabs sentiment values from probs dataframe
    dataframe_probs.reset_index(drop=True, inplace=True)
    df_pos['sentiment'] = df_probs['sentiment'].copy(deep=True)

    #Creates dataframe of only positive sentiment tweets
    df_pos = df_pos[df_pos['sentiment'] == 1]
    
    return df_pos

In [58]:
df_pos_uncleaned = create_pos_df(df, df_probs)

In [59]:
df_pos_cleaned = clean_tweets(df_pos_uncleaned)

In [60]:
def get_top_n_gram(corpus,ngram_range,n=None):

    #Vectorize corpus
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english', max_df=1.0, min_df=0.01).fit(corpus)

    #creates bag of words and finds the sums of words
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    
    #creates a list of words and their frequencies
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

    return words_freq[:n], bag_of_words

In [61]:
positives_and_count, pos_vec = get_top_n_gram(df_pos_cleaned['clean_text'], (1,1), 300)

positives = [x[0] for x in positives_and_count]
top_10_pos = positives[:10]
positives_minus_top_10 = [x for x in positives if x not in top_10_pos]
print(positives_minus_top_10)

['destiny2', 'threadsoflight', 'game', 'play', 'congrats', 'thanks', 'bring', 'destinythegame', 'make', 'really', 'don', 'hope', 'great', 'time', 'nice', 'better', 'new', 'day', 'playing', 'looks', 'congratulations', 'yeah', 'bungie', 'know', 'happy', 'trials', 'help', 'awesome', 'think', 've', 'going', 'dawning', 'wish', 'year', 'fun', 'got', 'space', 'hunter', 'people', 'shader', 'look', 'best', 'd2', 'live', 'crow', 'grandma', 'free', 'let', 'need', 'amazing', 'today', 'twab', 'friends', 'll', 'guardian', 'lol', 'week', 'use', 'getting', 'god', 'cool', 'pvp', 'changes', 'work', 'come', 'double', 'amp', 'right', 'looking', 'titan', 'team', 'maybe', 'ability', 'actually', 'oh', 'win', 'vex', 'pretty', 'flawless', 'thememe', 'super', 'man', 'played', 'rewards']


In [62]:
def extract_features(text, top_words):
    features = {}
    wordcount = 0

    #Adds 1 to wordcount if word in tokenized text is within the top list of words
    for sentence in sent_tokenize(text):
        for word in word_tokenize(sentence):
            if word.lower() in top_words:
                wordcount += 1

    #adds wordcount to dictionary
    features["wordcount"] = wordcount

    return features['wordcount']

features = [
    (extract_features(review, positives_minus_top_10))
    for review in df['clean_text']]

In [63]:
df_feat = pd.concat([df['clean_text'], pd.DataFrame(features,columns=['positive_word_count'])], axis=1)

In [64]:
# Train-test split
X_train,X_test,y_train,y_test = train_test_split(df_feat, df_probs['sentiment'],random_state=1)
X_train

,clean_text,positive_word_count
4319,read my mind,0
518,oh god please no more cookie baking please,2
4584,ahh thanks so much ghaul lt 3,1
3768,these cool down changes in combination with da...,2
2990,they talking about why eris had to put her r...,1
...,...,...
905,getting back into sniping slowly des...,3
5192,congratulations hell yeah,2
3980,ab you have been having problems for years...,0
235,dear eva could you kindly ask ada 1 to sel...,0


In [65]:
def get_top_n_gram_test(corpus, trained_corpus, ngram_range,n=None):

    #Vectorize corpus
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english', max_df=1.0, min_df=0.01).fit(trained_corpus)

    #Create bag of words and the sums
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 

    #Create a list of words and their frequencies
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n], bag_of_words

In [66]:
feat, feat_vec_train = get_top_n_gram(X_train['clean_text'], (1,1), 100)
feat, feat_vec_test = get_top_n_gram_test(X_test['clean_text'], X_train['clean_text'], (1,1), 100)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

#Create X_train vectorized
X_train_vec = pd.concat([X_train['positive_word_count'], pd.DataFrame(feat_vec_train.todense())], axis=1)
X_test_vec = pd.concat([X_test['positive_word_count'], pd.DataFrame(feat_vec_test.todense())], axis=1)

Baseline model is a simple logistic regression that scored 78% accuracy

In [70]:
log_model = LogisticRegression()
log_model.fit(X_train_vec, y_train)
log_model.score(X_test_vec, y_test)

0.7873015873015873